In [ ]:
molecule h2o2 {
  H     0.0000000000   0.9803530335  -0.8498671785
  O     0.0000000000   0.6988545188   0.0536419016
  O     0.0000000000  -0.6988545188   0.0536419016
  H     0.0000000000  -0.9803530335  -0.8498671785
}

# g_convergence needs to be tight for IRCs:
set g_convergence gau_verytight

set {
  basis                      dzp
  opt_type                   irc
  geom_maxiter               60
  irc_direction              backward
  d_convergence              9
  irc_points                 12
}

energy = frequencies('scf', dertype=1)

# Lower point group from C2v to C2
h2o2.reset_point_group('c2')

energy = optimize('scf')

In [2]:
# use psi4conda environment
import psi4
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import helpers as hp # Many useful functions from https://lcbc-epfl.github.io/iesm-public/intro.html

psi4.core.clean_options()

# The Z-matrix as a text string ### From previous optimization
data = """
       0 1
       H     0.0000000000   0.9803530335  -0.8498671785
       O     0.0000000000   0.6988545188   0.0536419016
       O     0.0000000000  -0.6988545188   0.0536419016
       H     0.0000000000  -0.9803530335  -0.8498671785

       symmetry c2
       units angstrom
       """ 

# Create the Molecule object
mol = psi4.geometry(data)             # Create Molecule object from data string

#hp.drawXYZ(mol)
hp.drawXYZ_labeled(mol)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:
ts = mol.clone()

psi4.core.clean_options()

output_file = "22x_ring_closing_TS1.log"
psi4.set_memory("2GB")
psi4.set_output_file(output_file, append=False, loglevel=20, print_header=True, inherit_loglevel=True, execute=True)
psi4.core.set_num_threads(4)

psi4.set_options({
        "OPT_TYPE": "ts",              # default => "min":  MIN, TS, IRC
        "FULL_HESS_EVERY": 0,         # default => -1 -> Does not do Hessian; 0 -> hessian at start
        "NORMAL_MODES_WRITE": True,    # default => False
    })


#energy_ts, hist = psi4.optimize("hf/sto-3g", molecule = ts, return_history=True)          
energy_ts, hist = psi4.optimize("b3pw91/6-31+G(d)", molecule = ts, return_history=True)   
#energy_ts, hist = psi4.optimize("b3lyp/6-311++G(d,p)", molecule = ts, return_history=True)   
#energy_ts, hist = psi4.optimize("B3LYP-d3bj2b/6-311++G(d,p)", molecule = ts, return_history=True)   


#print out TS in case something goes wrong
with open('ts.xyz', 'w') as f:
    f.write(ts.save_string_xyz_file())

Optimizer: Optimization complete!


In [6]:
!ls -alr *.molden_normal_modes

-rw-r--r--  1 blink  staff  16932 25 May 20:56 irc.C4H6.20360.molden_normal_modes
-rw-r--r--  1 blink  staff  16932 22 May 22:26 butane_TS2.C4H6.16013.molden_normal_modes
-rw-r--r--  1 blink  staff  34528 15 May 15:57 butane_TS.default.29456.molden_normal_modes
-rw-r--r--  1 blink  staff  34528 14 May 22:34 butane_TS.default.1122.molden_normal_modes
-rw-r--r--  1 blink  staff  18963 22 May 22:01 butane_TS.C4H6.16013.molden_normal_modes
-rw-r--r--  1 blink  staff   2937 30 May 20:55 22x_ring_closing_TS1.H2O2.56818.molden_normal_modes
-rw-r--r--  1 blink  staff  18963 28 May 21:48 10_ring_closing_TS1.C4H6.49499.molden_normal_modes
-rw-r--r--  1 blink  staff  18963 27 May 21:57 10_ring_closing_TS1.C4H6.29409.molden_normal_modes
-rw-r--r--  1 blink  staff  18963 25 May 23:27 10_ring_closing_TS1.C4H6.20360.molden_normal_modes


In [7]:
hp.show_normal_modes(filename='22x_ring_closing_TS1.H2O2.56818.molden_normal_modes')

interactive(children=(Dropdown(description='Normal mode:', options=((-944.9230467387, 0), (-641.3313884771, 1)…

In [16]:
output_file = "22_Butane_PES_1.out"
psi4.set_memory("4GB")
psi4.set_output_file(output_file, append=False, loglevel=20, print_header=True, inherit_loglevel=True, execute=True)
psi4.core.set_num_threads(4)

irc_direction = "forward"

mol3 = ts.clone()

psi4.set_options({
     "geom_maxiter":500,
     "full_hess_every":0,
     "opt_type":"irc",
     "irc_step_size":0.25,
     "ensure_bt_convergence":True,
     "irc_direction":irc_direction,
     "reference": "rhf",
     "g_convergence": "GAU_VERYTIGHT", # default => QCHEM:  MOLPRO, GAU, GAU_LOOSE, GAU_TIGHT, INTERFRAG_TIGHT, GAU_VERYTIGHT, TURBOMOLE, CFOUR, NWCHEM_LOOSE
     "IRC_POINTS": 100
     })

E, history = psi4.optimize('b3pw91/6-31+G(d)',molecule=mol3, return_history=True)

#energy_ts, hist = psi4.optimize("hf/sto-3g", molecule = ts, return_history=True)          
#energy_ts, hist = psi4.optimize("b3pw91/6-31+G(d)", molecule = ts, return_history=True)   
#energy_ts, hist = psi4.optimize("b3lyp/6-311++G(d,p)", molecule = ts, return_history=True)   
#energy_ts, hist = psi4.optimize("B3LYP-d3bj2b/6-311++G(d,p)", molecule = ts, return_history=True)   
print("Done")


Optimizer: Optimization complete!
[{'step_number': 0, 'q': [1.8369564698838443, 2.7424255475431325, 1.8369564698838443, 1.8410580442108186, 1.8410580442108186, 0.0], 'x': [[-4.871753492572071e-08, 1.8616501412023885, -1.6376268130782388], [6.619903487366585e-08, 1.3712127737715647, 0.13265007511626753], [-6.619903487366585e-08, -1.3712127737715647, 0.13265007511626753], [4.871753492572071e-08, -1.8616501412023885, -1.6376268130782388]], 'f_q': [-1.1252550299895521e-05, 0.00010141789239141331, -1.125255029989557e-05, 1.48037621351278e-05, 1.4803762135127969e-05, 3.529991611205609e-09], 'f_x': [-1.9927862834163867e-09, 4.762078027790977e-06, 1.2995676688311207e-05, 2.710037028469608e-09, 9.665581436358784e-05, -1.2995676688311146e-05, -2.710037028469608e-09, -9.665581436358777e-05, -1.299567668831134e-05, 1.9927862834163867e-09, -4.762078027791053e-06, 1.2995676688311278e-05], 'energy': -151.4727870722188, 'q_pivot': [1.8369566527322676, 2.742425353641491, 1.8369566527322676, 1.841058164

In [17]:

traj = hp.mol2traj(mol3, history["coordinates"])
hp.drawXYZGeomSlider(traj)

interactive(children=(IntSlider(value=0, description='conf', max=28), Output()), _dom_classes=('widget-interac…

In [14]:
history["coordinates"]

[array([[-0.03771485,  1.85559399, -1.60434431],
        [ 0.05284245,  1.31958595,  0.09936757],
        [-0.05284245, -1.31958595,  0.09936757],
        [ 0.03771485, -1.85559399, -1.60434431]]),
 array([[-0.01931311,  1.85986554, -1.62414151],
        [ 0.02727382,  1.32998304,  0.11916478],
        [-0.02727382, -1.32998304,  0.11916478],
        [ 0.01931311, -1.85986554, -1.62414151]]),
 array([[-0.02164893,  1.86171528, -1.62446112],
        [ 0.03050324,  1.32948919,  0.11948438],
        [-0.03050324, -1.32948919,  0.11948438],
        [ 0.02164893, -1.86171528, -1.62446112]]),
 array([[-0.02152118,  1.86167425, -1.62448030],
        [ 0.03032532,  1.32950252,  0.11950356],
        [-0.03032532, -1.32950252,  0.11950356],
        [ 0.02152118, -1.86167425, -1.62448030]]),
 array([[-0.02152785,  1.86167603, -1.62447934],
        [ 0.03033462,  1.32950176,  0.11950261],
        [-0.03033462, -1.32950176,  0.11950261],
        [ 0.02152785, -1.86167603, -1.62447934]]),
 array([[-